In [1]:
##되는 번호: 2,3,5,8,9,10,11,21  (좌표만 바꾸기)

In [ ]:
# import sys
import random
import numpy as np
import cv2

src = cv2.imread("C:/Users/Yunji Noh/OneDrive/image_processing/data/BC20.jpg")

if src is None:
    print('Image load failed!')
    sys.exit()

#src = cv2.resize(src, (0, 0), fx=0.5, fy=0.5)
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)


h, w = src.shape[:2]
dst1 = np.zeros((h, w, 3), np.uint8)
dst2 = np.zeros((h, w, 3), np.uint8)

# 이진화
_, src_bin = cv2.threshold(src_gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# 외곽선 검출
contours, _ = cv2.findContours(src_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

for i in range(len(contours)):
    pts = contours[i]

    c = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    cv2.drawContours(dst1, contours, i, c, 1)

    # 너무 작은 객체는 제외
    if (cv2.contourArea(pts) < 1000):
        continue

    # 외곽선 근사화
    approx = cv2.approxPolyDP(pts, cv2.arcLength(pts, True)*0.02, True)

    # 컨벡스가 아니면 제외
    if not cv2.isContourConvex(approx):
        continue

    if len(approx) == 4:
        cv2.drawContours(dst2, contours, i, c, 2)
        break  # 첫 번째로 검출된 외곽선만 사용하여 perspective transformation 수행

        
if len(approx) == 4:
    # 사각형의 네 꼭지점 좌표 추출
    points = approx.reshape(-1, 2).astype(np.float32)
    
    # 네 꼭지점 좌표에서 최소/최대 x, y 값을 구하여 w, h 계산
    min_x = np.min(points[:, 0])
    max_x = np.max(points[:, 0])
    min_y = np.min(points[:, 1])
    max_y = np.max(points[:, 1])
    
    w = 400
    h = 600

#- min_y, - min_x

    # 변환 후 좌표 설정
    dst_points = np.array([[0, h], [w, h],[w, 0], [0, 0]], dtype=np.float32)
    

    # 변환 행렬 계산
    transform_matrix = cv2.getPerspectiveTransform(points, dst_points)
    
    # 이미지 변환
    dst_image = cv2.warpPerspective(src, transform_matrix, (w, h))
    sharp = cv2.filter2D(dst_image, -1, np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]))
    
    
    
    # 결과 출력
    cv2.imshow('dst_image', dst_image)
   # cv2.imshow('src', src)
    #cv2.imshow('src_bin', src_bin)
    #cv2.imshow('dst1', dst1)
    cv2.imshow('dst2', dst2)
    cv2.imshow('sharp2', sharp)
    cv2.imwrite("C:/Users/Yunji Noh/OneDrive/image_processing/project_mid/result_proimg/sharp2.jpg", sharp)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
